In [1]:
print("hello")

hello


In [5]:
from autogen import UserProxyAgent

proxy = UserProxyAgent(name="proxy", code_execution_config={"executor": "ipython-embedded"})

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [2]:
import os

from autogen import ConversableAgent

agent = ConversableAgent(
    "chatbot",
    llm_config={"config_list": [{"model": "gpt-4o-mini", "api_key": os.environ["OPENAI_API_KEY"]}]},
    code_execution_config=False,  # Turn off code execution, by default it is off.
    function_map=None,  # No registered functions, by default it is None.
    human_input_mode="NEVER",  # Never ask for human input.
)

In [3]:
reply = agent.generate_reply(messages=[{"content": "Tell me a joke about California girls.", "role": "user"}])
print(reply)

Why did the California girl bring a ladder to the bar?

Because she heard the drinks were on the house!


In [7]:
agent_with_number = ConversableAgent(
    "agent_with_number",
    system_message="You are playing a game of guess-my-number. "
    "In the first game, you have the "
    "number 53 in your mind, and I will try to guess it. "
    "If I guess too high, say 'too high', if I guess too low, say 'too low'. ",
    llm_config={"config_list": [{"model": "gpt-4o-mini", "api_key": os.environ["OPENAI_API_KEY"]}]},
    max_consecutive_auto_reply=1,  # maximum number of consecutive auto-replies before asking for human input
    is_termination_msg=lambda msg: "61" in msg["content"],  # terminate if the number is guessed by the other agent
    human_input_mode="TERMINATE",  # ask for human input until the game is terminated
)

agent_guess_number = ConversableAgent(
    "agent_guess_number",
    system_message="I have a number in my mind, and you will try to guess it. "
    "If I say 'too high', you should guess a lower number. If I say 'too low', "
    "you should guess a higher number. ",
    llm_config={"config_list": [{"model": "gpt-4o-mini", "api_key": os.environ["OPENAI_API_KEY"]}]},
    human_input_mode="NEVER",
)

result = agent_with_number.initiate_chat(
    agent_guess_number,
    message="I have a number between 1 and 100. Guess it!",
)

agent_with_number (to agent_guess_number):

I have a number between 1 and 100. Guess it!

--------------------------------------------------------------------------------
agent_guess_number (to agent_with_number):

Is it 50?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
agent_with_number (to agent_guess_number):

Too low!

--------------------------------------------------------------------------------
agent_guess_number (to agent_with_number):

Is it 75?

--------------------------------------------------------------------------------
agent_with_number (to agent_guess_number):

How are you doing?

--------------------------------------------------------------------------------
agent_guess_number (to agent_with_number):

I'm here to help! Just let me know if you want to continue with the number guessing game or if you have other questions.

--------------------------------------------------------------------------------


In [1]:
import tempfile

from autogen import ConversableAgent
from autogen.coding import LocalCommandLineCodeExecutor

# Create a temporary directory to store the code files.
temp_dir = tempfile.TemporaryDirectory()

# Create a local command line code executor.
executor = LocalCommandLineCodeExecutor(
    timeout=10,  # Timeout for each code execution in seconds.
    work_dir=temp_dir.name,  # Use the temporary directory to store the code files.
)

# Create an agent with code executor configuration.
code_executor_agent = ConversableAgent(
    "code_executor_agent",
    llm_config=False,  # Turn off LLM for this agent.
    code_execution_config={"executor": executor},  # Use the local command line code executor.
    human_input_mode="ALWAYS",  # Always take human input for this agent for safety.
)

flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.


In [4]:
message_with_code_block = """This is a message with code block.
The code block is below:
```python
import numpy as np
import matplotlib.pyplot as plt
x = np.random.randint(0, 100, 100)
y = np.random.randint(0, 100, 100)
plt.scatter(x, y)
plt.savefig('scatter.png')
print('Scatter plot saved to scatter.png')
```
This is the end of the message.
"""

# Generate a reply for the given code.
reply = code_executor_agent.generate_reply(messages=[{"role": "user", "content": message_with_code_block}])
print(reply)


>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...
exitcode: 0 (execution succeeded)
Code output: Scatter plot saved to scatter.png



In [3]:
! pip install -qqq matplotlib numpy

In [5]:
import os

print(os.listdir(temp_dir.name))
# We can see the output scatter.png and the code file generated by the agent.

['tmp_code_e24bf32d4a21990fb9e4b5eb889ebe5a.py', 'scatter.png']


In [7]:
temp_dir.cleanup()

In [13]:
from typing import Dict, List
def fetch_restaurant_data(restaurant_name: str) -> Dict[str, List[str]]:
    # This function takes in a restaurant name and returns the reviews for that restaurant. 
    # The output should be a dictionary with the key being the restaurant name and the value being a list of reviews for that restaurant.
    # The "data fetch agent" should have access to this function signature, and it should be able to suggest this as a function call. 
    # Example:
    # > fetch_restaurant_data("Applebee's")
    # {"Applebee's": ["The food at Applebee's was average, with nothing particularly standing out.", ...]}
    restaurant_reviews = {}
    
    with open("restaurant-data.txt", "r") as f:
        lines = f.readlines()
        
    for line in lines:
        name = line.split(".", 1)[0]
        if name != restaurant_name:
            continue
        review = line.split(".", 1)[1]
        if name not in restaurant_reviews:
            restaurant_reviews[name] = []
        restaurant_reviews[name].append(review)
        
    return restaurant_reviews

In [15]:
fetch_restaurant_data("Taco Bell")

{'Taco Bell': [' The food at Taco Bell was bad, with flavors that seemed artificial. The customer service was average, neither particularly helpful nor rude.\n',
  ' The food was bad, with flavors that were off and textures that were strange. The service was average, neither particularly good nor terrible.\n',
  ' The food was bad and left me feeling queasy. The customer service was forgettable, neither good nor terrible.\n',
  ' The food at Taco Bell was bad, with flavors that seemed artificial and unappealing. The service was average, neither particularly good nor terrible.\n',
  ' The food was bad, but the service was average. The tacos were unpleasant and lacked flavor, though the staff was polite enough.\n',
  ' The food at Taco Bell was bad, with flavors that seemed artificial. The service was average, neither impressive nor terrible.\n',
  ' The food at Taco Bell was bad, with flavors that felt artificial and unappetizing. The customer service was average, neither adding to nor 

In [13]:
from typing import Dict, List, Annotated
import re
def fetch_restaurant_data(restaurant_name: Annotated[str, "The name of the restaurant"]) -> Dict[str, List[str]]:
    # This function takes in a restaurant name and returns the reviews for that restaurant. 
    # The output should be a dictionary with the key being the restaurant name and the value being a list of reviews for that restaurant.
    # The "data fetch agent" should have access to this function signature, and it should be able to suggest this as a function call. 
    # Example:
    # > fetch_restaurant_data("Applebee's")
    # {"Applebee's": ["The food at Applebee's was average, with nothing particularly standing out.", ...]}
    restaurant_reviews = {}
    restaurant_name = re.sub(r"[\s\-]", "", restaurant_name.lower())
    with open("restaurant-data.txt", "r") as f:
        lines = f.readlines()
        
    for line in lines:
        name = line.split(".", 1)[0]
        name = re.sub(r"[\s\-]", "", name.lower())
        print(name)
        if name != restaurant_name:
            continue
        review = line.split(".", 1)[1]
        if name not in restaurant_reviews:
            restaurant_reviews[name] = []
        restaurant_reviews[name].append(review)
        
    return restaurant_reviews

In [16]:
import os
from autogen import ConversableAgent
entrypoint_agent_system_message = "You are a helpful AI assistant. "
"You can help users find information about curtain restaurants in the database. " 
# example LLM config for the entrypoint agent
llm_config = {"config_list": [{"model": "gpt-4o-mini", "api_key": os.environ.get("OPENAI_API_KEY")}]}
# the main entrypoint/supervisor agent
data_fetch_agent = ConversableAgent(
    "Data Fetch Agent", 
    system_message=entrypoint_agent_system_message, 
    llm_config=llm_config,
    max_consecutive_auto_reply=1,
    human_input_mode="NEVER",  # Never ask for human input.
    silent = True
)
entrypoint_agent = ConversableAgent(
    name="User",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
    silent = True
)
data_fetch_agent.register_for_llm(name="fetch_reviews_for_restaurant", description="Fetches the reviews for a specific restaurant.")(fetch_restaurant_data)
entrypoint_agent.register_for_execution(name="fetch_reviews_for_restaurant")(fetch_restaurant_data)

<function __main__.fetch_restaurant_data(restaurant_name: Annotated[str, 'The name of the restaurant']) -> Dict[str, List[str]]>

In [17]:
chat_result = entrypoint_agent.initiate_chat(data_fetch_agent, message="What is the overall score for taco bell?", max_turns=2)


>>>>>>>> EXECUTING FUNCTION fetch_reviews_for_restaurant...


In [12]:
# Get the chat history.
import pprint
# pprint.pprint(chat_result.chat_history)
import json
restaurant_name = json.loads((chat_result.chat_history[1]
                   .get("tool_calls", [{}])[0]
                   .get("function", {})
                   .get("arguments", {})))
name = restaurant_name["restaurant_name"]
print(name)
reviews = json.loads(chat_result.chat_history[-1]["content"])
print(reviews)
# reviews = reviews[name]
# print(reviews)
# print(len(reviews))

Taco Bell
{'tacobell': [' The food at Taco Bell was bad, with flavors that seemed artificial. The customer service was average, neither particularly helpful nor rude.\n', ' The food was bad, with flavors that were off and textures that were strange. The service was average, neither particularly good nor terrible.\n', ' The food was bad and left me feeling queasy. The customer service was forgettable, neither good nor terrible.\n', ' The food at Taco Bell was bad, with flavors that seemed artificial and unappealing. The service was average, neither particularly good nor terrible.\n', ' The food was bad, but the service was average. The tacos were unpleasant and lacked flavor, though the staff was polite enough.\n', ' The food at Taco Bell was bad, with flavors that seemed artificial. The service was average, neither impressive nor terrible.\n', ' The food at Taco Bell was bad, with flavors that felt artificial and unappetizing. The customer service was average, neither adding to nor det

In [47]:
review_analysis_prompt = """
Analyze the restaurant review to extract two scores: food_score and customer_service_score, both from 1 to 5.

Look for two specific adjectives in the review:

	•	Food: adjectives that describe the food quality.
	•	Customer service: adjectives that describe the service quality.

Use the following scoring system based on the adjectives:

	•	1/5: awful, horrible, disgusting
	•	2/5: bad, unpleasant, offensive
	•	3/5: average, uninspiring, forgettable
	•	4/5: good, enjoyable, satisfying
	•	5/5: awesome, incredible, amazing

Output format:

	•	food_score: (score)
	•	customer_service_score: (score)

Example:

Review:

	“The food was average, but the customer service was unpleasant.”

Output:

	•	food_score: 3
	•	customer_service_score: 2

Now, analyze this review:
"""

review_analysis_agent = ConversableAgent(
    name="Review Analysis Agent",
    system_message=review_analysis_prompt,
    llm_config=llm_config,
    max_consecutive_auto_reply=1,
    human_input_mode="NEVER",
)

review_1 = reviews[0]
review_result = entrypoint_agent.initiate_chat(review_analysis_agent, message=review_1)
result = review_result.chat_history[1]["content"]
print(result)

User (to Review Analysis Agent):

 The food at Taco Bell was bad, with flavors that seemed artificial. The customer service was average, neither particularly helpful nor rude.


--------------------------------------------------------------------------------


Review Analysis Agent (to User):

	•	food_score: 2
	•	customer_service_score: 3

--------------------------------------------------------------------------------
User (to Review Analysis Agent):



--------------------------------------------------------------------------------
	•	food_score: 2
	•	customer_service_score: 3


In [37]:
import math
def calculate_overall_score(restaurant_name: str, food_scores: List[int], customer_service_scores: List[int]) -> Dict[str, float]:
    # TODO
    # This function takes in a restaurant name, a list of food scores from 1-5, and a list of customer service scores from 1-5
    # The output should be a score between 0 and 10, which is computed as the following:
    # SUM(sqrt(food_scores[i]**2 * customer_service_scores[i]) * 1/(N * sqrt(125)) * 10
    # The above formula is a geometric mean of the scores, which penalizes food quality more than customer service. 
    # Example:
    # > calculate_overall_score("Applebee's", [1, 2, 3, 4, 5], [1, 2, 3, 4, 5])
    # {"Applebee's": 5.04}
    # NOTE: be sure to round the score to 2 decimal places.
    if len(food_scores) != len(customer_service_scores):
        raise ValueError("The number of food scores and customer service scores must be the same.")
    
    n = len(food_scores)
    score = 0
    
    for i in range(n):
        score += math.sqrt(food_scores[i]**2 * customer_service_scores[i])
        
    score *= 1/(n * math.sqrt(125)) * 10
    
    return {restaurant_name: round(score, 2)}

In [42]:
scoring_prompt = """
        Calculate the overall score for the restaurant based on the food and customer service scores.
    """

scoring_agent = ConversableAgent(
    name="Scoring Agent",
    system_message=scoring_prompt,
    llm_config=llm_config,
    max_consecutive_auto_reply=1,
    human_input_mode="NEVER",
)

scoring_agent.register_for_llm(name="calculate_overall_score", description="Calculate the overall score based on the review analysing results.")(calculate_overall_score)
entrypoint_agent.register_for_execution(name="calculate_overall_score")(calculate_overall_score)

<function __main__.calculate_overall_score(restaurant_name: str, food_scores: List[int], customer_service_scores: List[int]) -> Dict[str, float]>

In [52]:
score_result = entrypoint_agent.initiate_chat(scoring_agent, message=name + ": " + result, max_turns=2)
print(score_result.chat_history[-1]["content"])
score = json.loads(score_result.chat_history[-1]["content"]).get(name)
print(score)
score += 0.01
print(score)

User (to Scoring Agent):

Taco Bell: 	•	food_score: 2
	•	customer_service_score: 3

--------------------------------------------------------------------------------
Scoring Agent (to User):

***** Suggested tool call (call_R6vOVPanLOKqaIMFYqFwAyDi): calculate_overall_score *****
Arguments: 
{"restaurant_name":"Taco Bell","food_scores":[2],"customer_service_scores":[3]}
****************************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION calculate_overall_score...
User (to Scoring Agent):

User (to Scoring Agent):

***** Response from calling tool (call_R6vOVPanLOKqaIMFYqFwAyDi) *****
{"Taco Bell": 3.1}
**********************************************************************

--------------------------------------------------------------------------------
{"Taco Bell": 3.1}
3.1
3.11


In [53]:
import diskcache

# 清除缓存
cache = diskcache.Cache('cache')
cache.clear()

0